In [1]:
import requests
import json
import os
import foursquare
import pandas as pd
from tqdm import tqdm

# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

#### Step 1: Set code to review full API

In [2]:
# Setup Foursquare API credentials 
api_key = os.environ.get('FOURSQUARE_API_KEY')

# Define the endpoint URL for the Foursqare request
endpoint = "https://api.foursquare.com/v3/places/search"

# Define the query parameters to include only the latitude and longitude fields
def foursquare_query(latitude, longitude):
    params = {
        'll': f'{latitude},{longitude}', 'radius':'1000'
        }
    #Headers specific to FourSquare - used to specify wanted metadata - not typically found under all APIs. Found under https://location.foursquare.com/developer/reference/authentication
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
        }

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content



#### Step 2: Retrieve Foursquare results

In [3]:
#Retrieve the Foursquare query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = foursquare_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Foursquare query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, location_name):
    json_content = foursquare_query(latitude, longitude)
    poi_list = []
    for poi in json_content['results']:
        poi_dict = {}
        poi_dict['fsq_id'] = poi['fsq_id']
        poi_dict['location_name'] = location_name
        poi_dict['poi_type'] = [category['name'] for category in poi['categories']]
        poi_list.append(poi_dict)
    return poi_list

#### Step 3: Parse through data to retrieve point of interest (POI) details (Available from Foursquare - location, type)

In [4]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

rand_df = bike_df.sample(n=250)

# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df['name'].unique()
latitudes = rand_df['latitude'].unique()
longitudes = rand_df['longitude'].unique()


# create an empty list to store the dictionaries
data_list = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for fsq_id in fsq_ids:
        data_list.append({'fsq_id': fsq_id['fsq_id'], 'location_name': fsq_id['location_name'], 'poi_type': fsq_id['poi_type']})

100%|██████████| 250/250 [03:12<00:00,  1.30it/s]


#### Step 4:  Put your parsed results into a DataFrame

In [38]:
POI_CityBikes_DF = pd.DataFrame(data_list)

# Yelp

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

##### Set code to review full API

In [13]:
# Setup Yelp API credentials 
api_key = os.environ.get('YELP_API_KEY')

# Define the endpoint URL for the Yelp request
endpoint = "https://api.yelp.com/v3/businesses/search"

# Define the query parameters to include only the latitude and longitude fields
def yelp_query(latitude, longitude):
    params = {
        'll': f'{latitude},{longitude}', 'radius':'1000'
        }
    #Headers specific to Yelp - used to specify wanted metadata - not typically found under all APIs.
    headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [16]:
#Retrieve the Yelp query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = yelp_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Yelp query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, names):
    json_content = yelp_query(latitude, longitude)
    poi_list = []
    for poi in json_content['results']:
        poi_dict = {}
        poi_dict['id'] = poi['id']
        poi_dict['location'] = location
        # poi_dict['poi_type'] = [category['title'] for category in poi['categories']]
        poi_dict['categories'] = poi['categories']
        poi_dict['rating'] = poi['rating']
        poi_dict['price'] = poi('price')
        poi_list.append(poi_dict)
    return poi_list

In [17]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

rand_df = rand_df

# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df['name'].unique()
latitudes = rand_df['latitude'].unique()
longitudes = rand_df['longitude'].unique()


# create an empty list to store the dictionaries
data_list = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for fsq_id in fsq_ids:
        data_list.append({'fsq_id': fsq_id['fsq_id'], 'location_name': fsq_id['location_name'], 'poi_type': fsq_id['poi_type']})

  0%|          | 0/2 [00:00<?, ?it/s]


KeyError: 'results'

In [9]:
# bike_radius('51.529163', '-0.109971')
latitude = 51.529163
longitude = -0.109971


import json

json_content = bike_radius_info(latitude, longitude, location name)
print(json.dumps(json_content, indent=4))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3531325255.py, line 8)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating